In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import datetime
import pandas_datareader as dr

#Import Model Packages 
from sklearn.cluster import KMeans, AgglomerativeClustering,AffinityPropagation, DBSCAN
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold

from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD

#Other Helper Packages and functions
import matplotlib.ticker as ticker
from itertools import cycle

#Diable the warnings
import warnings
warnings.filterwarnings('ignore')




Dataset http://www2.informatik.uni-freiburg.de/~cziegler/BX/
Book crossing dataset 


In [4]:
# Load dataset
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataset = read_csv('/content/drive/My Drive/SP500Data.csv',index_col=0)

In [ ]:
# type(dataset)

pandas.core.frame.DataFrame

In [5]:
# Book  
books = pd.read_csv('/content/drive/My Drive/BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1",low_memory=False)

books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

# Users
users = pd.read_csv('/content/drive/My Drive/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1",low_memory=False)

users.columns = ['userID', 'Location', 'Age']


# Rating
ratings = pd.read_csv('/content/drive/My Drive/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1",low_memory=False)

ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\nSkipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\nSkipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\nSkipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [6]:
ratings.head()

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
users.head()

,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
# Merge data 

# Book rating
combine_book_rating = pd.merge(ratings,books,on='ISBN')

columns = ['yearOfPublication', 'publisher', 'bookAuthor','imageUrlS','imageUrlM','imageUrlL']

combine_book_rating = combine_book_rating.drop(columns,axis=1)

combine_book_rating.head()

,userID,ISBN,bookRating,bookTitle
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel


DATA PROCESSING 

In [10]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

In [11]:
book_ratingCount = (combine_book_rating.
     groupby(by = ['bookTitle'])['bookRating'].
     count().
     reset_index().
     rename(columns = {'bookRating': 'totalRatingCount'})
     [['bookTitle', 'totalRatingCount']]
    )


In [12]:
book_ratingCount.head()

,bookTitle,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [13]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, 
                                                         left_on = 'bookTitle', 
                                                         right_on = 'bookTitle', 
                                                         how = 'left')
rating_with_totalRatingCount.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,60
1,2313,034545104X,5,Flesh Tones: A Novel,60
2,6543,034545104X,0,Flesh Tones: A Novel,60
3,8680,034545104X,5,Flesh Tones: A Novel,60
4,10314,034545104X,9,Flesh Tones: A Novel,60


In [15]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   241071.00
mean         4.28
std         16.74
min          1.00
25%          1.00
50%          1.00
75%          3.00
max       2502.00
Name: totalRatingCount, dtype: float64


In [16]:
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

0.90    7.00
0.91    8.00
0.92    9.00
0.93   10.00
0.94   11.00
0.95   13.00
0.96   16.00
0.97   20.00
0.98   29.00
0.99   50.00
Name: totalRatingCount, dtype: float64


In [17]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,60
1,2313,034545104X,5,Flesh Tones: A Novel,60
2,6543,034545104X,0,Flesh Tones: A Novel,60
3,8680,034545104X,5,Flesh Tones: A Novel,60
4,10314,034545104X,9,Flesh Tones: A Novel,60


In [18]:
rating_popular_book.count

<bound method DataFrame.count of          userID        ISBN  ...                   bookTitle totalRatingCount
0        276725  034545104X  ...        Flesh Tones: A Novel               60
1          2313  034545104X  ...        Flesh Tones: A Novel               60
2          6543  034545104X  ...        Flesh Tones: A Novel               60
3          8680  034545104X  ...        Flesh Tones: A Novel               60
4         10314  034545104X  ...        Flesh Tones: A Novel               60
...         ...         ...  ...                         ...              ...
1029947  274061  1551663147  ...                       Fever               68
1030363  274808  0449701913  ...                  Homecoming              137
1030824  275970  0865714215  ...              Stormy Weather              103
1030868  275970  1586210661  ...      Me Talk Pretty One Day              457
1031042  276563  0027197107  ...  The Cat Who Went to Heaven               55

[288740 rows x 5 columns]>

In [19]:
combined = rating_popular_book.merge(users, left_on = 'userID', right_on = 'userID', how = 'left')

# Extract location 
us_canada_user_rating = combined[combined['Location'].str.contains("usa|canada")]

us_canada_user_rating=us_canada_user_rating.drop('Age', axis=1)

us_canada_user_rating.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount,Location
0,276725,034545104X,0,Flesh Tones: A Novel,60,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,60,"cincinnati, ohio, usa"
2,6543,034545104X,0,Flesh Tones: A Novel,60,"strafford, missouri, usa"
3,8680,034545104X,5,Flesh Tones: A Novel,60,"st. charles county, missouri, usa"
4,10314,034545104X,9,Flesh Tones: A Novel,60,"beaverton, oregon, usa"


In [20]:
us_canada_user_rating.shape

(251615, 6)

In [24]:
print('Initial dataframe shape {0}'.format(us_canada_user_rating.shape))
if not us_canada_user_rating[us_canada_user_rating.duplicated(['userID', 'bookTitle'])].empty:
    initial_rows = us_canada_user_rating.shape[0]

    print('Initial dataframe shape {0}'.format(us_canada_user_rating.shape))

    us_canada_user_rating = us_canada_user_rating.drop_duplicates(['userID', 'bookTitle'])

    current_rows = us_canada_user_rating.shape[0]

    print('New dataframe shape {0}'.format(us_canada_user_rating.shape))

    print('Removed {0} rows'.format(initial_rows - current_rows))


Initial dataframe shape (248949, 6)


In [25]:
us_canada_user_rating_pivot = us_canada_user_rating.pivot(index = 'bookTitle', columns = 'userID', values = 'bookRating').fillna(0)

us_canada_user_rating_matrix = csr_matrix(us_canada_user_rating_pivot.values)

In [26]:
us_canada_user_rating_pivot

userID,8,9,14,16,17,23,26,32,39,42,44,51,56,67,75,77,95,97,99,114,125,132,133,135,144,165,169,178,185,199,207,224,226,228,230,243,244,254,256,272,...,278550,278552,278556,278561,278582,278586,278601,278633,278641,278645,278648,278653,278663,278672,278683,278694,278723,278732,278740,278769,278771,278773,278781,278786,278796,278798,278800,278807,278813,278819,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
bookTitle,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
16 Lighthouse Road,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1984,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1st to Die: A Novel,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2010: Odyssey Two,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Zoya,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"\O\"" Is for Outlaw""",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0

In [29]:
us_canada_user_rating_matrix

<2442x40017 sparse matrix of type '<class 'numpy.float64'>'
	with 97607 stored elements in Compressed Sparse Row format>

In [36]:
us_canada_user_rating_pivot.shape

(2442, 40017)

In [30]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

model_knn.fit(us_canada_user_rating_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [43]:
# 1
query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])

distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
#distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[1907, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
        #print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[1907]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for The Joy Luck Club:

1: The Kitchen God's Wife, with distance of 0.8924720231590498:
2: The Hundred Secret Senses, with distance of 0.8925939736968544:
3: H Is for Homicide (Kinsey Millhone Mysteries (Paperback)), with distance of 0.9184359634796739:
4: Reader, with distance of 0.9267287428495845:
5: Plague Dogs, with distance of 0.9276392973005907:


In [44]:
us_canada_user_rating_pivot2 = us_canada_user_rating.pivot(index = 'userID', columns = 'bookTitle', values = 'bookRating').fillna(0)

In [45]:
us_canada_user_rating_pivot2.head()

bookTitle,10 Lb. Penalty,16 Lighthouse Road,1984,1st to Die: A Novel,2010: Odyssey Two,204 Rosewood Lane,2061: Odyssey Three,24 Hours,2nd Chance,3rd Degree,4 Blondes,50 Simple Things You Can Do to Save the Earth,84 Charing Cross Road,A 2nd Helping of Chicken Soup for the Soul (Chicken Soup for the Soul Series (Paper)),A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Clockwork Orange (Norton Paperback Fiction),A Cold Heart (Alex Delaware Novels (Paperback)),A Cold Heart: An Alex Delaware Novel,A Confederacy of Dunces,A Confederacy of Dunces (Evergreen Book),A Cry In The Night,A Cup of Tea (Ballantine Reader's Circle),A Dangerous Fortune,A Darkness More Than Night,A Day Late and a Dollar Short,A Density of Souls,A Fine Balance,"A Game of Thrones (A Song of Ice and Fire, Book 1)",A Girl Named Zippy: Growing Up Small in Mooreland Indiana (Today Show Book Club #3),A Great Deliverance,A Heartbreaking Work Of Staggering Genius : A Memoir Based on a True Story,A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Kiss of Shadows (Meredith Gentry Novels (Paperback)),A Knight in Shining Armor,A Lesson Before Dying (Vintage Contemporaries (Paperback)),...,Wild Justice,Wild Mountain Thyme,Wild Swans: Three Daughters of China,Windmills of the Gods,Windy City Blues (V.I. Warshawski Novels (Paperback)),Wings,Winter Moon,Winter Prey,Winter Solstice,"Winter's Heart (The Wheel of Time, Book 9)",Winter's Tale,Wish You Well,Without Remorse,Witness in Death (Eve Dallas Mysteries (Paperback)),"Wizard and Glass (The Dark Tower, Book 4)","Wizard's First Rule (Sword of Truth, Book 1)",Wobegon Boy,Woman on the Edge of Time,Women Who Run with the Wolves,Women in His Life,Women of the Silk : A Novel,"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",World of Pies : A Novel,Wouldn't Take Nothing for My Journey Now,Writ of Execution,Writing Down the Bones,Wuthering Heights,Wuthering Heights (Penguin Classics),Wuthering Heights (Wordsworth Classics),Xenocide (Ender Wiggins Saga (Paperback)),YOU BELONG TO ME,Year of Wonders,You Belong To Me,You Shall Know Our Velocity,Young Wives,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",stardust
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0

In [46]:
us_canada_user_rating_pivot2.shape

(40017, 2442)

In [55]:
X = us_canada_user_rating_pivot2.values.T
X.shape

(2442, 40017)

In [54]:
us_canada_user_rating_pivot2.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(2442, 12)

In [57]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(2442, 2442)

In [63]:
us_canada_book_title = us_canada_user_rating_pivot2.columns

us_canada_book_list = list(us_canada_book_title)

coffey_hands = us_canada_book_list.index("The Green Mile: Night Journey (Green Mile Series)")

#Green Mile: Night Journey (Green Mile Series)
#The Green Mile: Coffey's Hands (Green Mile Series)
print(coffey_hands)

1907


In [62]:
us_canada_book_title = us_canada_user_rating_pivot2.columns
us_canada_book_title.shape

list(us_canada_book_title)

['10 Lb. Penalty',
 '16 Lighthouse Road',
 '1984',
 '1st to Die: A Novel',
 '2010: Odyssey Two',
 '204 Rosewood Lane',
 '2061: Odyssey Three',
 '24 Hours',
 '2nd Chance',
 '3rd Degree',
 '4 Blondes',
 '50 Simple Things You Can Do to Save the Earth',
 '84 Charing Cross Road',
 'A 2nd Helping of Chicken Soup for the Soul (Chicken Soup for the Soul Series (Paper))',
 'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
 'A Bend in the Road',
 'A Case of Need',
 'A Child Called \\It\\": One Child\'s Courage to Survive"',
 'A Civil Action',
 'A Clockwork Orange (Norton Paperback Fiction)',
 'A Cold Heart (Alex Delaware Novels (Paperback))',
 'A Cold Heart: An Alex Delaware Novel',
 'A Confederacy of Dunces',
 'A Confederacy of Dunces (Evergreen Book)',
 'A Cry In The Night',
 "A Cup of Tea (Ballantine Reader's Circle)",
 'A Dangerous Fortune',
 'A Darkness More Than Night',
 'A Day Late and a Dollar Short',
 'A Density of Souls',
 'A Fine Balance',
 'A Game of T

In [64]:
corr_coffey_hands  = corr[coffey_hands]

In [71]:
list(us_canada_book_title[(corr_coffey_hands<1.0) & (corr_coffey_hands>0.85)])

['BAG OF BONES : A NOVEL',
 'Bag of Bones',
 'Carrie',
 'Christine',
 'Cold Fire',
 'Desperation',
 'Different Seasons',
 'Different Seasons (Signet)',
 'Dragonsdawn',
 'Dreamcatcher',
 'Four Past Midnight',
 "Gerald's Game",
 'Hearts In Atlantis : New Fiction',
 'Insomnia',
 'It',
 'Misery',
 'Needful Things',
 'Needful Things: The Last Castle Rock Story',
 'Nightmares &amp; Dreamscapes',
 'Pet Sematary',
 'Rose Madder',
 "Salem's Lot",
 'Shipping News',
 'Skeleton Crew',
 'Stand',
 'Strangers',
 "Tailchaser's Song",
 'The Bachman Books: Rage, the Long Walk, Roadwork, the Running Man',
 'The Dark Half',
 'The Dead Zone',
 'The Face of Fear',
 'The Girl Who Loved Tom Gordon',
 'The Girl Who Loved Tom Gordon : A Novel',
 'The Green Mile: Coffey on the Mile (Green Mile Series)',
 "The Green Mile: Coffey's Hands (Green Mile Series)",
 'The Green Mile: The Bad Death of Eduard Delacroix (Green Mile Series)',
 'The Green Mile: The Complete Serial Novel',
 'The Green Mile: The Mouse on the Mi

In [73]:
recommend1 = us_canada_book_title[(corr_coffey_hands<1.0) & (corr_coffey_hands>0.85)]

In [74]:
recommend1

Index(['BAG OF BONES : A NOVEL', 'Bag of Bones', 'Carrie', 'Christine',
       'Cold Fire', 'Desperation', 'Different Seasons',
       'Different Seasons (Signet)', 'Dragonsdawn', 'Dreamcatcher',
       'Four Past Midnight', 'Gerald's Game',
       'Hearts In Atlantis : New Fiction', 'Insomnia', 'It', 'Misery',
       'Needful Things', 'Needful Things: The Last Castle Rock Story',
       'Nightmares &amp; Dreamscapes', 'Pet Sematary', 'Rose Madder',
       'Salem's Lot', 'Shipping News', 'Skeleton Crew', 'Stand', 'Strangers',
       'Tailchaser's Song',
       'The Bachman Books: Rage, the Long Walk, Roadwork, the Running Man',
       'The Dark Half', 'The Dead Zone', 'The Face of Fear',
       'The Girl Who Loved Tom Gordon',
       'The Girl Who Loved Tom Gordon : A Novel',
       'The Green Mile: Coffey on the Mile (Green Mile Series)',
       'The Green Mile: Coffey's Hands (Green Mile Series)',
       'The Green Mile: The Bad Death of Eduard Delacroix (Green Mile Series)',
       

In [78]:
lis1 = recommend1.to_list

In [87]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(lis1)

<bound method IndexOpsMixin.tolist of Index(['BAG OF BONES : A NOVEL', 'Bag of Bones', 'Carrie', 'Christine',
       'Cold Fire', 'Desperation', 'Different Seasons',
       'Different Seasons (Signet)', 'Dragonsdawn', 'Dreamcatcher',
       'Four Past Midnight', 'Gerald's Game',
       'Hearts In Atlantis : New Fiction', 'Insomnia', 'It', 'Misery',
       'Needful Things', 'Needful Things: The Last Castle Rock Story',
       'Nightmares &amp; Dreamscapes', 'Pet Sematary', 'Rose Madder',
       'Salem's Lot', 'Shipping News', 'Skeleton Crew', 'Stand', 'Strangers',
       'Tailchaser's Song',
       'The Bachman Books: Rage, the Long Walk, Roadwork, the Running Man',
       'The Dark Half', 'The Dead Zone', 'The Face of Fear',
       'The Girl Who Loved Tom Gordon',
       'The Girl Who Loved Tom Gordon : A Novel',
       'The Green Mile: Coffey on the Mile (Green Mile Series)',
       'The Green Mile: Coffey's Hands (Green Mile Series)',
       'The Green Mile: The Bad Death of Eduard D